In [6]:
%matplotlib ipympl
SCOPETYPE = "OPENADC"
PLATFORM = "CWHUSKY"
# SS_VER = "SS_VER_2_1"

%run "../Setup_Generic.ipynb"


import chipwhisperer as cw
from chipwhisperer.capture.scopes.OpenADC import OpenADC
from chipwhisperer.capture.targets.SimpleSerial2 import SimpleSerial2
from chipwhisperer.capture.api.programmers import XMEGAProgrammer
import matplotlib.pyplot as plt
import time
from pprint import pprint
import numpy as np
from typing import Optional, List, Set, Any
print(type(scope))
print(type(target))
print(type(prog))
scope: OpenADC = scope
target: SimpleSerial2 = target
prog: XMEGAProgrammer = prog

print("Scope:\n", scope)

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2     

In [7]:
refresh_rate = 100e6

scope.vglitch_setup("hp")

scope.adc.timeout = 5
# scope.adc.samples = 118_000
# print(scope.glitch.enabled)
# scope.glitch.enabled = True
# scope.clock.clkgen_src = "system" 
scope.clock.adc_mul = 1
scope.clock.clkgen_freq = refresh_rate

scope.glitch.output = "enable_only"
# print("'Real freq': scope.clock.adc_freq:", scope.clock.adc_freq)
scope.glitch.trigger_src = 'manual'
if scope.clock.adc_freq is not None:
    print(scope.clock.adc_freq / 1e6, "MHz")

# scope.io.glitch_hp = True
# scope.io.glitch_lp = True

print(scope.glitch.phase_shift_steps)

(ChipWhisperer Scope WARNING|File ChipWhispererHuskyClock.py:703) Target clock may drop; you may need to reset your target.


100.0 MHz
336


In [8]:
for a in dir(scope.glitch):
    v = getattr(scope.glitch, a)
    if not a.startswith('_') and not callable(v):
        print(f"{a}: {v}")

(ChipWhisperer Glitch ERROR|File ChipWhispererGlitch.py:1063) N/A for Husky
(ChipWhisperer Glitch ERROR|File ChipWhispererGlitch.py:1038) N/A for Husky


actual_num_glitches: 0
arm_timing: after_scope
clk_src: pll
cwg: <chipwhisperer.capture.scopes.cwhardware.ChipWhispererGlitch.ChipWhispererGlitch object at 0x10557ebb0>
enabled: True
ext_offset: 0
mmcm_locked: True
num_glitches: 1
offset: 0
offset_fine: 0
output: enable_only
phase_shift_steps: 336
pll: pll_src                       = xtal
adc_freq                      = 100000000.0
target_freq                   = 100000000.0
adc_mul                       = 1
pll_locked                    = True
adc_delay                     = 0
target_delay                  = 0
f_pfd                         = 12000000.0
f_vco                         = 2400000000.0
input divider                 = 1
input multiplier              = 50
feedback prescaler            = 4
prescaler                     = 4
output divider (target clock) = 6

repeat: 1
state: idle
trigger_src: manual
width: 0
width_fine: 0


In [10]:
scope.arm()
scope.glitch.repeat = 350

i = 0
while True:
    scope.glitch.repeat += 1
    i += 1
    time.sleep(3)
    scope.glitch.manual_trigger()
    print("Triggered", i, "With repeat:", scope.glitch.repeat)

Triggered 1 With repeat: 351
Triggered 2 With repeat: 352


KeyboardInterrupt: 